# 定义

`PyTorch` 是一种深度学习**框架**，包含 `Tensor` 数据结构、自动求导、神经网络(nn)、优化器(optim)、数据加载(utils.data)、设备(GPU加速)


## PyTorch 特性
* 动态计算图
* 自动微分
* 张量计算
* 丰富的API

## Ternsor
`Tensor` 是一个多维数组，跟 `ndarray` 类似，但支持GPU加速、自动微分和动态计算图计算。

组成部分：
* 数据：以一维数组的形式存储数据，通过 `shape` 和 `stride` 属性映射到多维视图中。
* 形状：`shape` 表示张量的维度，例如（3，224，244）表示三通道，224x224大小的张量，可以通过`torch.shape` 或者 `torch.size()` 查看
* `Dtype`：数据类型，例如：torch.float32,torch.int64，可以通过 `torch.dtype` 查看，在创建时可以指定参数类型
* 设备：通过 `device` 参数设定
* 梯度跟踪：布尔值，表示是否追踪该 Tensor 的梯度，通过`tensor.requires_grad` 查看或者设置


## PyTorch、Torch、Tensor的关系
PyTorch是一个框架，基于 `Torch` 库实现，也可以认为是一个大的库。

`Tensor` 是一个具体的数据结构

## Pytorch 和 Numpy 的关系

Numpy是python中一个科学计算基本库，提供了一个多维数组对象 `ndarray` 主要用于数值计算，不支持自动求导，仅支持CPU计算

Tensor 和 Numpy 可以相互转换，但需要注意设备位置（需要在CPU上），转换方法：

1. .numpy():将 tensor 转换为 numpy.
2. torch.from_numpy(): 将 numpy 转换为 tensor。注意这种转换方式是指两者共享相同的内存，一个改变另一个也会改变
3. torch.tensor(): 将 numpy 转换为 tensor，返回的Tensor和原来的numpy数据不再共享内存。所以计算速度慢

## Tensor 常用计算
1. 创建操作
   1. 从 Python 列表创建
   2. 创建全零或者全1张量
   3. 随机初始化
2. 数学运算
   1. 逐元素运算
   2. 矩阵乘法
   3. 聚合操作
3. 形状操作
   1. 改变形状
   2. 转置
4. 自动微分
   1. 启动梯度
   2. 计算梯度

### 创建tensor

torch.Tensor()是指创建一个浮点数的 tensor， 等同于 torch.FloatTensor()，类似的还有 torch.IntTensor, torch.LongTensor(), torch.DoubleTensor().不推荐使用

torch.tensor()需要在里面指定 tensor 类型。推荐使用

In [3]:
import torch
import torch.nn.functional as F

In [4]:
a = torch.Tensor([0, 1])
# a = torch.Tensor([[0,1],[2,0]], dtype=torch.int64)  错误，torch1.8版本以后，Tensor不支持指定类型
b = torch.FloatTensor([0, 1])
c = torch.tensor([[0, 1], [2, 0]], dtype=torch.float32) # 推荐

print(a.type(), b.type(), c.type())

d = torch.arange(0, 4)
e = torch.linspace(0, 10, steps=5)  # steps 表示分割次数
print(d,e)

torch.FloatTensor torch.FloatTensor torch.FloatTensor
tensor([0, 1, 2, 3]) tensor([ 0.0000,  2.5000,  5.0000,  7.5000, 10.0000])


In [5]:
# 生成特殊张量
zeros = torch.zeros(2,3)
ones = torch.ones(2, 3)
print(zeros)
print(ones)

tensor([[0., 0., 0.],
        [0., 0., 0.]])
tensor([[1., 1., 1.],
        [1., 1., 1.]])


In [6]:
# 随机初始化
rand = torch.rand(2, 3)  # 标准均匀分布随机值
print(rand)
randn = torch.rand(2, 3)   # 标准正态分布随机值
print(randn)

tensor([[0.6639, 0.6776, 0.4407],
        [0.6362, 0.5896, 0.1015]])
tensor([[0.8827, 0.1971, 0.7692],
        [0.0270, 0.4255, 0.3934]])


### 数学运算
* 逐元素运算
* 矩阵乘法
* 聚合运算

In [7]:
# 逐元素运算
a + b  # 加法（等价于 torch.add(a, b)）
a * b  # 逐元素乘法

tensor([0., 1.])

In [8]:
# 矩阵乘法
matmul = torch.matmul(a, b)
# 或者使用 @
matmul = a @ b

In [9]:
# 聚合运算
sum = torch.sum(a)  # 默认所有数据求和，可以设置维度
print(sum)
print(c)
mean = torch.mean(c,dim=0)
print(mean)

tensor(1.)
tensor([[0., 1.],
        [2., 0.]])
tensor([1.0000, 0.5000])


### 改变形状
1. 改变形状
   1. torch.view 不改变张量的数据，只是改变 shape 和 stride，但要求张量必须是连续的，如果不连续，需要调用 `torch.contiguous` 方法使张量连续,在内存连续的情况下，view性能更高效
   2. torch.reshape 在张量内存连续时，与view相同，不复制数据，如果，张量不连续，会复制数据。优点在于能够自动处理不连续的张量
2. 转置
   1. 经过转置后，张量在内存中不连续
   2. .T 适用于二维矩阵
   3. torch.transpose() 交换张量两个指定的维度，与原张量共享数据
   4. torch.permute() 同时调整多个维度

In [10]:
# 改变形状
reshaped = c.view(4,1)
print(reshaped)
reshaped[0] = 2
print(c)
print(reshaped.is_contiguous())  # True 表示连续

tensor([[0.],
        [1.],
        [2.],
        [0.]])
tensor([[2., 1.],
        [2., 0.]])
True


In [11]:
# 转置
c = torch.arange(0,12).view(4,3)
transposed = c.T
print(c,c.stride())
transpesed = torch.transpose(c, 0, 1)
print(transpesed, transpesed.is_contiguous(), transpesed.stride())

tensor([[ 0,  1,  2],
        [ 3,  4,  5],
        [ 6,  7,  8],
        [ 9, 10, 11]]) (3, 1)
tensor([[ 0,  3,  6,  9],
        [ 1,  4,  7, 10],
        [ 2,  5,  8, 11]]) False (1, 3)


### 自动微分相关
1. 启动梯度跟踪
2. 计算梯度

In [12]:
# 启动梯度跟踪
x = torch.tensor([1.0], requires_grad=True)

In [13]:
# 计算梯度
y = x ** 2
num = y.backward()
print(x.grad)

tensor([2.])


### 其他常用函数

In [14]:
# one-hot 编码
a = torch.arange(4)
a = F.one_hot(a, 4)
print(a)

tensor([[1, 0, 0, 0],
        [0, 1, 0, 0],
        [0, 0, 1, 0],
        [0, 0, 0, 1]])
